# Кластеризация чеков
Автор: _Хасанов Расим 11-804_ <br>
Хорошая статья про кластеризацию данных с word2vec: https://dylancastillo.co/nlp-snippets-cluster-documents-using-word2vec/

## Импорт необходимых библиотек

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import *
from sklearn.model_selection import train_test_split
import enchant
import nltk
from nltk.corpus import stopwords
import re
import string
import urllib
from pymystem3 import Mystem
import gensim
from gensim.models import word2vec
from sklearn.cluster import MiniBatchKMeans
from sklearn.metrics import silhouette_samples, silhouette_score
#nltk.download('popular')


mystem = Mystem()
ru_dict = enchant.Dict('ru_RU')
en_dict = enchant.Dict('en_US')

## Загрузка данных
Данные можно скачать по [ссылке](https://github.com/ydooG/receipts_clustering/tree/master/core/static/core/data/receipts.xlsx?raw=true)

In [2]:
raw_data = pd.read_excel('core/static/core/data/receipts.xlsx')
raw_data.sample(n=10)

,Unnamed: 0,id,protocolVersion,operationType,cashTotalSum,shiftNumber,counterSubmissionSum,totalSum,ecashTotalSum,nds18,...,items.nds10,items.nds18,items.paymentAgentByProductType,items.modifiers,userProperty.key,userProperty.value,propertiesUser.propertyValue,propertiesUser.propertyName,propertiesUser.key,propertiesUser.value
82,82,778608317,NaN,1,0,14,NaN,150000,150000,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
400,400,778611175,NaN,1,0,388,NaN,11500,11500,1917.0,...,NaN,NaN,NaN,NaN,NaN,NaN,d731322a22cad4dbc661c90633fabc0d,trust_purchase_token,NaN,NaN
1013,1013,666908702,NaN,1,0,20,NaN,10000,10000,1667.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
701,701,666902941,NaN,1,134700,218,NaN,134700,0,22450.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1091,1091,666909447,NaN,1,9900,777,NaN,9900,0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
624,624,666901769,NaN,1,0,113,NaN,3099,3099,517.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
743,743,666904573,NaN,1,0,21,NaN,22147,22147,1142.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
641,641,666902953,NaN,1,0,33,NaN,22000,22000,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
872,872,666906228,2.0,1,20000,416,0.0,20000,0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
774,774,666905259,NaN,1,0,3,NaN,50000,50000,8333.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Ручной анализ датасета
Так как задача состоит в кластеризации наименований товаров, то стоит рассматривать только названия товаров. Цену товара тоже не имеет особого смысла, так как: "Картофель за 50р/кг", тот же самый, что и "Картофель за 70р/кг".<br><br>
Таким образом задача сводится к области NLP. То есть: определить похожесть/различность слов

In [3]:
df = raw_data[['items.name']]
for row in df['items.name']:
    print(row)

Электроснабжение
Электроснабжение
Аванс за услуги связи 9091082396
Аванс за услуги связи 9092626865
Аванс за услуги связи 9089895687
Оплата проезда за наличный расчет
Расчет с участником азартной игры
Продление домена zhabinskiy.me на 1 год по тарифу Эконом
Лиц.на ПО: Рег.домена польз.в нац.реестрах зарубежн.стран (ссTLD)
ГВС в целях содержания о.и МКД
Содержание и текущий ремонт общего имущества
ХВС в целях содержания о.и МКД
Э/Э в целях содержания о.и МКД
Перевозка пассажиров и багажа
Чипсы Lays Молодой Зеленый лук 50г фл/п :28 Фрито Лей,шт
Напиток сок натуральный б/газ б/алк,шт
Напиток сок натуральный б/газ б/алк,шт
Стакан 200мл пластик 1шт 1/100,шт
Стакан 200мл пластик 1шт 1/100,шт
Напиток CC Coca-Cola газ 0,5л п/б :24 Coca-Cola,шт
Хачапури с сыром,93
КПГ
Сигареты Bond Street Compact Blue
ТРК   1    Бензин АИ-95-К5
Сок innocent мультифруктовый п
Хлеб для тостов с отрубями Про
Сыр творожный с огурцами и зел
Шоколадные батончики Kinder Fe
Пакет майка Азбука Вкуса 30х57
Оплата проезда

## Препроцессинг данных

In [4]:
df.value_counts()

items.name                                                 
Услуги связи                                                   49
Лицензия на использование ПО сайта checkyour.name              28
Электроснабжение                                               22
Перевозка пассажиров и багажа                                  20
Поездка за наличные                                            18
                                                               ..
Вода ШИШКИН ЛЕС минеральная газированная 1 л                    1
Вода мин Ергалах б/г 1,5л пэт                                   1
Вода минеральная "Суздальские напитки" Родниковая газ 600мл     1
Вода минеральная Шифа 1,5л ПЭТ                                  1
чай пакет Гринфилд в асс                                        1
Length: 809, dtype: int64

В данных 809 уникальных значений. Среди них: номера телефонов, даты, и прочие идентификационные номера. От них стоит избавиться.

Далее я написал кастомный токенизатор:
- убирает бессмысленные пробелы (двойные, тройные и т.д.)
- убирает знаки препинания
- убирает цифры
- убирает стоп-слова
- убирает слова, состоящие из менее чем 3-ех букв
- пытается расшифровать аббревиатуры (библиотека от яндекса [mystem](https://yandex.ru/dev/mystem/))
- лемматизирует слова (библиотека от яндекса [mystem](https://yandex.ru/dev/mystem/))

In [5]:
noise = stopwords.words('russian') + ['\n']
def custom_tokenizer(text):
    text = re.sub('[{}]'.format(string.punctuation), ' ', text)
    text = re.sub(r"\d+", " ", text)
    text = mystem.lemmatize(text)
    result = []
    for word in text:
        word = ''.join(word.split())
        if word not in noise and len(word) > 2:
            result.append(word)
    return result

In [6]:
custom_tokenizer('Аванс за услуги связи 9067186262')

['аванс', 'услуга', 'связь']

## Обучение модели на имеющихся словах в датасете

In [7]:
sentences = []
for sentence in df['items.name']:
    tokenized_sent = custom_tokenizer(sentence)
    if tokenized_sent not in sentences:
        sentences.append(tokenized_sent)
sentences

[['электроснабжение'],
 ['аванс', 'услуга', 'связь'],
 ['оплата', 'проезд', 'наличный', 'расчет'],
 ['расчет', 'участник', 'азартный', 'игра'],
 ['продление', 'домен', 'zhabinskiy', 'год', 'тариф', 'эконом'],
 ['лицо',
  'рег',
  'домен',
  'польза',
  'нац',
  'реестр',
  'зарубежн',
  'страна',
  'ссTLD'],
 ['ГВС', 'цель', 'содержание', 'МКД'],
 ['содержание', 'текущий', 'ремонт', 'общий', 'имущество'],
 ['ХВС', 'цель', 'содержание', 'МКД'],
 ['цель', 'содержание', 'МКД'],
 ['перевозка', 'пассажир', 'багаж'],
 ['чипсы', 'Lays', 'молодой', 'зеленый', 'лук', 'фритый', 'лить'],
 ['напиток', 'сок', 'натуральный', 'газ', 'алк'],
 ['стакан', 'пластик'],
 ['напиток', 'Coca', 'Cola', 'газ', 'Coca', 'Cola'],
 ['хачапури', 'сыр'],
 ['КПГ'],
 ['сигарета', 'Bond', 'Street', 'Compact', 'Blue'],
 ['трк', 'бензин'],
 ['сок', 'innocent', 'мультифруктовый'],
 ['хлеб', 'тост', 'отруби'],
 ['сыр', 'творожный', 'огурец', 'зел'],
 ['шоколадный', 'батончик', 'Kinder'],
 ['пакет', 'майка', 'азбука', 'вкус'

In [8]:
model = word2vec.Word2Vec(sentences, workers=4, vector_size=300, min_count=1, window=10, sample=1e-3)

Рассмотрим, насколько модель хорошо ищет схожие слова, на примере "яблоко", "услуга", "электроснабжение"

In [9]:
model.wv.most_similar("яблоко", topn=3)

[('ланч', 0.18598251044750214),
 ('прозра', 0.18013416230678558),
 ('вкусна', 0.1486368328332901)]

In [10]:
model.wv.most_similar("услуга", topn=3)

[('дуб', 0.2125006914138794),
 ('чай', 0.16697664558887482),
 ('освежий', 0.1653933972120285)]

In [11]:
model.wv.most_similar("электроснабжение", topn=3)

[('фиолетовый', 0.18135391175746918),
 ('ZBW', 0.1764535754919052),
 ('танеко', 0.16647504270076752)]

Как можем видеть, модель даёт очень маленькие вероятности сходства слов ( < 20% ). Это говорит о скудном количестве слов в датасете. Соответственно, на новых данных модель работать не будет.
Попробуем использовать предобученную модель на внешних данных 

In [12]:
def vectorize(list_of_docs, model):
    """Generate vectors for list of documents using a Word Embedding

    Args:
        list_of_docs: List of documents
        model: Gensim's Word Embedding

    Returns:
        List of document vectors
    """
    features = []

    for tokens in list_of_docs:
        zero_vector = np.zeros(model.vector_size)
        vectors = []
        for token in tokens:
            if token in model.wv:
                try:
                    vectors.append(model.wv[token])
                except KeyError:
                    continue
        if vectors:
            vectors = np.asarray(vectors)
            avg_vec = vectors.mean(axis=0)
            features.append(avg_vec)
        else:
            features.append(zero_vector)
    return features

def mbkmeans_clusters(
    X, 
    k, 
    mb, 
    print_silhouette_values=True, 
    ):
    """Generate clusters and print Silhouette metrics using MBKmeans

    Args:
        X: Matrix of features.
        k: Number of clusters.
        mb: Size of mini-batches.
        print_silhouette_values: Print silhouette values per cluster.

    Returns:
        Trained clustering model and labels based on X.
    """
    km = MiniBatchKMeans(n_clusters=k, batch_size=mb).fit(X)
    print(f"For n_clusters = {k}")
    print(f"Silhouette coefficient: {silhouette_score(X, km.labels_):0.2f}")
    print(f"Inertia:{km.inertia_}")

    if print_silhouette_values:
        sample_silhouette_values = silhouette_samples(X, km.labels_)
        print(f"Silhouette values:")
        silhouette_values = []
        for i in range(k):
            cluster_silhouette_values = sample_silhouette_values[km.labels_ == i]
            silhouette_values.append(
                (
                    i,
                    cluster_silhouette_values.shape[0],
                    cluster_silhouette_values.mean(),
                    cluster_silhouette_values.min(),
                    cluster_silhouette_values.max(),
                )
            )
        silhouette_values = sorted(
            silhouette_values, key=lambda tup: tup[2], reverse=True
        )
        for s in silhouette_values:
            print(
                f"    Cluster {s[0]}: Size:{s[1]} | Avg:{s[2]:.2f} | Min:{s[3]:.2f} | Max: {s[4]:.2f}"
            )
    return km, km.labels_

In [13]:
vectorized_sents = vectorize(sentences, model=model)
len(vectorized_sents), len(vectorized_sents[0])

(543, 300)

In [14]:
clusters_num = 10
clustering, cluster_labels = mbkmeans_clusters(
	X=vectorized_sents,
    k=clusters_num,
    mb=500,
)
df_clusters = pd.DataFrame({
    "text": sentences,
    "tokens": [" ".join(text) for text in sentences],
    "cluster": cluster_labels
})

For n_clusters = 10
Silhouette coefficient: -0.03
Inertia:0.2056469060953916
Silhouette values:
    Cluster 1: Size:3 | Avg:0.57 | Min:0.54 | Max: 0.62
    Cluster 6: Size:16 | Avg:0.17 | Min:0.03 | Max: 0.31
    Cluster 9: Size:14 | Avg:0.12 | Min:0.01 | Max: 0.22
    Cluster 5: Size:48 | Avg:0.06 | Min:-0.03 | Max: 0.13
    Cluster 7: Size:1 | Avg:0.00 | Min:0.00 | Max: 0.00
    Cluster 0: Size:16 | Avg:-0.01 | Min:-0.06 | Max: 0.07
    Cluster 3: Size:98 | Avg:-0.05 | Min:-0.11 | Max: -0.00
    Cluster 4: Size:218 | Avg:-0.05 | Min:-0.17 | Max: -0.01
    Cluster 2: Size:52 | Avg:-0.06 | Min:-0.12 | Max: 0.00
    Cluster 8: Size:77 | Avg:-0.06 | Min:-0.11 | Max: 0.00


In [15]:
print("Most representative terms per cluster (based on centroids):")
for i in range(clusters_num):
    tokens_per_cluster = ""
    most_representative = model.wv.most_similar(positive=[clustering.cluster_centers_[i]], topn=5)
    for t in most_representative:
        tokens_per_cluster += f"{t[0]} "
    print(f"Cluster {i}: {tokens_per_cluster}")

Most representative terms per cluster (based on centroids):
Cluster 0: яблоко пирожок бумага белый финест 
Cluster 1: МКД цель содержание мяс расчет 
Cluster 2: вода услуга батон банан ассорти 
Cluster 3: напиток хлеб газ вода ХВС 
Cluster 4: пиво разовый мята жевательный колонка 
Cluster 5: сигарета Drive винстон CLUB Impulse 
Cluster 6: пакет майка сигарета мини Parker 
Cluster 7: бутылка импульс мельник наушник капельный 
Cluster 8: трк кофе капучино американо томат 
Cluster 9: блю винстон сигарета хсенс гарнир 


In [16]:
test_cluster = 8
most_representative_docs = np.argsort(
    np.linalg.norm(vectorized_sents - clustering.cluster_centers_[test_cluster], axis=1)
)
for d in most_representative_docs[:3]:
    print(sentences[d])
    print("-------------")

[]
-------------
['сигарета', 'маркиров', 'некст', 'DUBLISS', 'VIOLET', 'MIX', 'фильтр', 'супертонкий', 'капсула', 'ментол', 'твердый', 'упак', 'МРЦ', 'руб']
-------------
['вставной', 'наушник', 'Xiaomi', 'AirDots', 'Pro', 'True', 'Wireless', 'Earphones', 'белый', 'TWSEJ', 'ZBW']
-------------


## Обучение модели на внешних данных

In [17]:
#urllib.request.urlretrieve("http://rusvectores.org/static/models/rusvectores2/ruscorpora_mystem_cbow_300_2_2015.bin.gz", "ruscorpora_mystem_cbow_300_2_2015.bin.gz")

In [18]:
model_path = 'ruscorpora_mystem_cbow_300_2_2015.bin.gz'
model2 = gensim.models.KeyedVectors.load_word2vec_format(model_path, binary=True)


In [19]:
vectorized_sents2 = vectorize(sentences, model=model2)
vectorized_sents2

AttributeError: 'KeyedVectors' object has no attribute 'wv'

In [ ]:
type(vectorized_sents2[0])

In [ ]:
clusters_num = 10
clustering, cluster_labels = mbkmeans_clusters(
	X=vectorized_sents2,
    k=clusters_num,
    mb=500,
)
df_clusters = pd.DataFrame({
    "text": sentences,
    "tokens": [" ".join(text) for text in sentences],
    "cluster": cluster_labels
})